In [1]:
#importing the necessary libraries
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from PIL import Image
from torchvision.models import ResNet50_Weights, resnet51

In [2]:
# Download and preprocess Food-101 dataset
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Download Food-101 dataset
train_dataset = datasets.Food101(root='./data', split='train', download=True, transform=data_transforms['train'])
val_dataset = datasets.Food101(root='./data', split='test', download=True, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layer to match the Food-101 classes (101 classes)
model.fc = nn.Linear(model.fc.in_features, 101)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=0.001)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Training the model
num_epochs = 3 # Adjust as necessary
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


Epoch 1/3 - Loss: 2.3619, Accuracy: 43.42%
Epoch 2/3 - Loss: 1.9304, Accuracy: 52.30%
Epoch 3/3 - Loss: 1.8266, Accuracy: 54.48%


In [5]:
# Saving model to pytorch
from pathlib import Path

MODEL_PATH = Path("Models_01")
MODEL_PATH.mkdir(parents = True, exist_ok = True)

MODEL_NAME = "img_classifier3.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to : {MODEL_SAVE_PATH}")
torch.save(obj = model.state_dict(),
           f = MODEL_SAVE_PATH)


Saving model to : Models_01\img_classifier3.pth


In [6]:
import torch
import torchvision.models as models

# Load your fine-tuned model (e.g., trained on Food-101)
loaded_model = models.resnet50(pretrained=False)  # Replace this with your fine-tuned model
loaded_model.fc = torch.nn.Linear(loaded_model.fc.in_features, 101)
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
loaded_model.to(device) #Load the model to CUDA

# Count the total number of layers
def count_layers(loaded_model):
    total_layers = 0
    for name, module in loaded_model.named_modules():
        if isinstance(module, torch.nn.Module):  # Count layers that are of type nn.Module
            total_layers += 1
    return total_layers

total_layers = count_layers(loaded_model)
print(f"Total number of layers in the ResNet-50 model: {total_layers}")


C:\Users\aarya\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aarya\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\aarya\AppData\Local\Temp\ipykernel_10768\3478123888.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future 

Total number of layers in the ResNet-50 model: 151


In [7]:
next(loaded_model.parameters()).device

device(type='cuda', index=0)

In [8]:
# Evaluating the model
loaded_model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = loaded_model(inputs)
        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy:.2f}%")

Validation Accuracy: 58.83%


In [3]:
from pathlib import Path
import torch
from torchvision import models

# Step 1: Define paths
MODEL_PATH = Path("Models_01")
MODEL_NAME = "img_classifier3.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Step 2: Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = models.resnet50(pretrained=False)  # Define the architecture
loaded_model.fc = torch.nn.Linear(loaded_model.fc.in_features, 101)  # Adjust for your dataset
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH))  # Load the saved weights
loaded_model.to(device)  # Move the model to the appropriate device

# Step 3: Set to evaluation mode
loaded_model.eval()

print("Model loaded successfully and ready for inference!")


C:\Users\aarya\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aarya\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\aarya\AppData\Local\Temp\ipykernel_27300\1012580704.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future

Model loaded successfully and ready for inference!


In [4]:
def predict_image(image_path, model, transform, labels):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    loaded_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = loaded_model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_label = labels[predicted.item()]
        
    return predicted_label

# Example of using the model to predict an image
image_path = r"C:\Users\aarya\Desktop\FoodImages\pizza.jpg"  # Replace with your image path
predicted_label = predict_image(image_path, loaded_model, data_transforms['val'], val_dataset.classes)
print(f"Predicted Label: {predicted_label}")

Predicted Label: pizza


In [6]:
import os

# Set the environment variable within the notebook
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/aarya/Desktop/JSON_key/food-nonfood-classifier-f184cf461aa9.json"

In [7]:
# Verify that the environment variable is set correctly
print("Credential Path:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))


Credential Path: C:/Users/aarya/Desktop/JSON_key/food-nonfood-classifier-f184cf461aa9.json


In [ ]:
from google.cloud import vision
from PIL import Image
import torch

# Vision API Function
def is_food_image(image_path):
    """Check if the image contains food using Google Vision API."""
    client = vision.ImageAnnotatorClient()

    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Perform label detection
    response = client.label_detection(image=image)
    labels = response.label_annotations

    # Check if any label indicates food
    for label in labels:
        if "food" in label.description.lower() or "dish" in label.description.lower():
            return True
    return False

# Prediction Function
def predict_image(image_path, model, transform, labels):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_label = labels[predicted.item()]
        
    return predicted_label

# Example of using the model to predict an image
image_path = r"C:\Users\aarya\Desktop\FoodImages\pizza.jpg"  # Replace with your image path

# Check if the image is food
if is_food_image(image_path):
    predicted_label = predict_image(image_path, loaded_model, data_transforms['val'], val_dataset.classes)
    print(f"Predicted Label: {predicted_label}")
else:
    print("The uploaded image does not appear to be food. Please upload a valid food image.")


Predicted Label: pizza


In [14]:
import requests
import json

API_KEY = 'thrhrDUfhV02DVJunGVEUvA1eqPlbuZNUMurNUxS'  # Replace with your actual API key
BASE_URL = "https://api.nal.usda.gov/fdc/v1/"

def get_food_data(food_name):
    search_url = f"{BASE_URL}foods/search?query={food_name}&api_key={API_KEY}"
    response = requests.get(search_url)
    
    if response.status_code == 200:
        data = response.json()
        if data['foods']:
            food_data = data['foods'][0]
            food_id = food_data['fdcId']
            detail_url = f"{BASE_URL}food/{food_id}?api_key={API_KEY}"
            detail_response = requests.get(detail_url)
            if detail_response.status_code == 200:
                detail_data = detail_response.json()
                
                # Print the whole response to inspect it
                print(json.dumps(detail_data, indent=4))
                
                # Extract nutritional information
                nutrients = detail_data.get('foodNutrients', [])
                nutrition_info = {
                    "food_name": food_data['description'],
                    "calories": None,
                    "protein": None,
                    "fat": None,
                    "carbs": None
                }

                for nutrient in nutrients:
                    # Check if 'nutrientName' exists before accessing it
                    if 'nutrientName' in nutrient:
                        if nutrient['nutrientName'] == 'Energy':
                            nutrition_info["calories"] = nutrient.get('value', 'N/A')
                        elif nutrient['nutrientName'] == 'Protein':
                            nutrition_info["protein"] = nutrient.get('value', 'N/A')
                        elif nutrient['nutrientName'] == 'Total lipid (fat)':
                            nutrition_info["fat"] = nutrient.get('value', 'N/A')
                        elif nutrient['nutrientName'] == 'Carbohydrate, by difference':
                            nutrition_info["carbs"] = nutrient.get('value', 'N/A')
                
                return nutrition_info
        else:
            print("Food item not found.")
    else:
        print("Error in the API request.")
    return None

# Example usage
food_name = "pizza"
food_details = get_food_data(food_name)
if food_details:
    print(json.dumps(food_details, indent=4))


{
    "discontinuedDate": "",
    "foodComponents": [],
    "foodAttributes": [
        {
            "id": 3168362,
            "value": 5,
            "name": "Nutrient Added"
        },
        {
            "id": 3168363,
            "value": 6,
            "name": "Nutrient Removed"
        }
    ],
    "foodPortions": [],
    "fdcId": 2629362,
    "description": "PIZZA",
    "publicationDate": "8/31/2023",
    "foodNutrients": [
        {
            "type": "FoodNutrient",
            "nutrient": {
                "id": 1093,
                "number": "307",
                "name": "Sodium, Na",
                "rank": 5800,
                "unitName": "mg"
            },
            "foodNutrientDerivation": {
                "id": 70,
                "code": "LCCS",
                "description": "Calculated from value per serving size measure"
            },
            "id": 32910044,
            "amount": 461.0
        },
        {
            "type": "FoodNutrient",
      